# BEE 4750 Final Project Code

**Name**: 

**ID**: 

> **Due Date**
>


## Setup


In [ ]:
import Pkg
Pkg.activate(".")
Pkg.instantiate()
Pkg.add(["Distributions", "JuMP", "HiGHS", "Plots", "StatsBase"])
using JuMP
using HiGHS
using Distributions
using DataFrames
using StatsBase
using Plots
using Random


# ---------------------------------------
# USER / MODEL PARAMETERS (editable)
# ---------------------------------------


# Hydraulic / loading
Q_m3d     = 100000.0      # m3/d
N_in_mgL  = 40.0         # mg N / L
N_load_kgd = Q_m3d * N_in_mgL / 1000.0   # kg N / d


# Effluent requirement
TN_target_mgL = 10.0
N_eff_max_kgd = Q_m3d * TN_target_mgL / 1000.0 # kg N / d allowed


# Reference removal efficiencies (at T_ref)
T_ref = 20.0                # °C (reference temperature)
η_PNA_ref  = 0.85           # fraction of influent N removed by PN/A train at T_ref (per-unit of flow routed)
η_CONV_ref = 0.90           # fraction of influent N removed by conventional train at T_ref


# Temperature coefficient (theta per °C)
# Use a modest temperature sensitivity: per-degree multiplier
# so η(T) = η_ref * theta_temp^(T - T_ref)
theta_temp = 1.015   # ~1.5% performance increase per °C (adjustable)


# Energy intensities (kWh per kg N removed) at T_ref
E_PNA_ref  = 1.6    # kWh/kgN
E_CONV_ref = 4.5    # kWh/kgN


# COD demand for denitrification (kg COD / kg N removed) at T_ref
COD_CONV_factor_ref = 4.0


# Costs
cost_energy_per_kWh = 0.12   # $/kWh
cost_COD_per_kg      = 0.45  # $/kg COD
carbon_price_per_tCO2e = 50.0 # $ per tonne CO2e (social + regulatory cost) - used to price GHG in objective


# new parameters (add near top)
capex_PNA_total = 2000000.0   # $ total installed cost for PN/A (example)
economic_lifetime_years = 20.0
discount_rate = 0.06            # 6% nominal discount rate


# compute annualized capital cost via CRF then convert to $/day
crf = (discount_rate * (1 + discount_rate)^economic_lifetime_years) / ((1 + discount_rate)^economic_lifetime_years - 1)
capex_PNA_annual = capex_PNA_total * crf
capex_PNA_daily = capex_PNA_annual / 365.0


# N2O emission factors (kg N2O-N per kg N removed) - literature ranges; adjust as desired
ef_N2O_per_kgN_PNA  = 0.002  # PN/A-specific N2O-N per kg N removed
ef_N2O_per_kgN_CONV = 0.010  # conventional pathway higher in many cases (example)


# Global Warming Potential (100-yr)
GWP_N2O = 273.0  # approximate GWP100 for N2O (unitless) - multiply N2O mass (kg) by this to get kg CO2e
# Note: We'll convert from N2O-N mass (kg N2O-N) to N2O mass (kg) by factor (44/28)


# GHG cap (kg CO2e per day). If <= 0, no cap is enforced.
# Example: set to a very large number to effectively disable cap
GHG_cap = Inf   # set to Float64 value if you want a hard cap, e.g. 1e6


# Binary build cost weight: if you want to bias toward building or not, adjust via capex
# (we've already added capex_PNA_annual to objective when y_pna = 1)


# Monte Carlo sampling for temperature
n_runs = 100
T_mean = 15.0      # mean °C
T_std  = 3.0       # standard deviation °C
rng = MersenneTwister(42)


temps = rand(rng, Normal(T_mean, T_std), n_runs)


# Storage for results
results = DataFrame(run = 1:n_runs,
                    T = temps,
                    pct_PNA = zeros(n_runs),
                    objective = zeros(n_runs),
                    total_energy_kWh = zeros(n_runs),
                    total_COD_kg = zeros(n_runs),
                    total_GHG_CO2e_kg = zeros(n_runs),
                    built_PNA = zeros(Int, n_runs)
)


# ---------------------------------------
# Helper function: build & solve MILP for a single temperature
# ---------------------------------------
function solve_milp_for_temperature(T;
    verbose=false)


    # temperature multipliers
    temp_factor = theta_temp^(T - T_ref)


    # Effective efficiencies at this temperature (we clip between 0 and 0.995)
    η_PNA = clamp(η_PNA_ref * temp_factor, 1e-6, 0.995)
    η_CONV = clamp(η_CONV_ref * temp_factor, 1e-6, 0.995)


    # Energy & COD factors assumed proportional to N removed (we keep them constant here but you could vary with T)
    E_PNA = E_PNA_ref    # kWh per kgN removed
    E_CONV = E_CONV_ref


    COD_CONV_factor = COD_CONV_factor_ref


    # Build model
    model = Model(HiGHS.Optimizer)
    set_silent(model)  # comment out if you want solver output


    # Decision variables
    @variable(model, 0 <= x_pna <= 1, base_name="x_pna")   # fraction of flow routed to PN/A
    @variable(model, 0 <= x_conv <= 1, base_name="x_conv")
    @variable(model, y_pna, Bin)                          # binary: build PN/A (1) or not (0)


    # Continuous operational variables - expressed as dependent variables via constraints
    @variable(model, rem_PNA >= 0)   # kg N/d removed by PN/A
    @variable(model, rem_CONV >= 0)  # kg N/d removed by conventional


    @variable(model, aer_PNA_kWh >= 0)
    @variable(model, aer_CONV_kWh >= 0)
    @variable(model, COD_CONV_kg >= 0)


    # Flow split
    @constraint(model, x_pna + x_conv == 1.0)


    # If PN/A not built then x_pna == 0
    # enforce x_pna <= y_pna  (big-M style by 1 since x_pna is <=1)
    @constraint(model, x_pna <= y_pna)


    # Nitrogen removal relationships (linear)
    # rem_PNA = η_PNA * x_pna * N_load_kgd
    @constraint(model, rem_PNA == η_PNA * x_pna * N_load_kgd)
    @constraint(model, rem_CONV == η_CONV * x_conv * N_load_kgd)


    # Effluent N constraint: N_load - (rem_PNA + rem_CONV) <= N_eff_max_kgd
    @constraint(model, N_load_kgd - (rem_PNA + rem_CONV) <= N_eff_max_kgd)


    # Aeration linearization
    @constraint(model, aer_PNA_kWh == E_PNA * rem_PNA)
    @constraint(model, aer_CONV_kWh == E_CONV * rem_CONV)


    # COD requirement for conventional
    @constraint(model, COD_CONV_kg == COD_CONV_factor * rem_CONV)


    # GHG calculations (N2O pathway)
    # N2O-N mass (kg) per day:
    # n2oN_PNA_kg = ef_N2O_per_kgN_PNA * rem_PNA
    # convert to N2O mass: N2O_mass = n2oN_kg * (44/28)
    # CO2e_kg = N2O_mass * GWP_N2O
    @expression(model, n2oN_PNA_kg, ef_N2O_per_kgN_PNA * rem_PNA)
    @expression(model, n2oN_CONV_kg, ef_N2O_per_kgN_CONV * rem_CONV)


    @expression(model, N2O_mass_kg, (n2oN_PNA_kg + n2oN_CONV_kg) * (44.0/28.0))
    @expression(model, GHG_CO2e_kg, N2O_mass_kg * GWP_N2O / 1000.0 * 1000.0)
    # Explanation: N2O_mass_kg * GWP_N2O gives kg CO2e; (we keep as kg CO2e)
    # Above we wrote /1000*1000 to be explicit but it's identity; kept for clarity


    # Optional GHG cap constraint (if finite)
    if isfinite(GHG_cap)
        @constraint(model, GHG_CO2e_kg <= GHG_cap)
    end


    # Objective: minimize operational cost + chemical cost + GHG social cost + annualized capex if PNA built
    # operational energy cost:
    @expression(model, energy_cost, cost_energy_per_kWh * (aer_PNA_kWh + aer_CONV_kWh))
    @expression(model, cod_cost, cost_COD_per_kg * COD_CONV_kg)


    # carbon (GHG) cost: convert kg CO2e to tonnes CO2e then multiply by carbon price
    @expression(model, ghg_cost, carbon_price_per_tCO2e * (GHG_CO2e_kg / 1000.0))


    # capital cost added only if y_pna == 1 (linearization - use y_pna * capex)
    @expression(model, capex_cost, capex_PNA_daily * y_pna)


    @objective(model, Min, (energy_cost + cod_cost + ghg_cost) + capex_cost)


    optimize!(model)


    status = termination_status(model)
    if verbose
        println("Status: ", status)
    end


    # collect outputs
    out = Dict()
    out[:status] = status
    out[:x_pna] = value(x_pna)
    out[:x_conv] = value(x_conv)
    out[:y_pna] = (round(value(y_pna))) #used to say int
    out[:rem_PNA] = value(rem_PNA)
    out[:rem_CONV] = value(rem_CONV)
    out[:aer_PNA_kWh] = value(aer_PNA_kWh)
    out[:aer_CONV_kWh] = value(aer_CONV_kWh)
    out[:COD_CONV_kg] = value(COD_CONV_kg)
    out[:GHG_CO2e_kg] = value(GHG_CO2e_kg)
    out[:objective] = objective_value(model)


    return out
end


# ---------------------------------------
# Monte Carlo loop
# ---------------------------------------
println("Running Monte Carlo MILP with n_runs = $n_runs ...")
for i in 1:n_runs
    T = temps[i]
    sol = solve_milp_for_temperature(T, verbose=false)


    pct_pna = sol[:x_pna] * 100.0
    results.pct_PNA[i] = pct_pna
    results.objective[i] = sol[:objective]
    results.total_energy_kWh[i] = sol[:aer_PNA_kWh] + sol[:aer_CONV_kWh]
    results.total_COD_kg[i] = sol[:COD_CONV_kg]
    results.total_GHG_CO2e_kg[i] = sol[:GHG_CO2e_kg]
    results.built_PNA[i] = sol[:y_pna]
end


# ---------------------------------------
# Quick summary prints
# ---------------------------------------
println("\nSummary of Monte Carlo runs (n = $n_runs):")
println("Temperature (mean, std): ", mean(results.T), " °C, ", std(results.T), " °C")
println("Percent of runs where PN/A was built (y_pna == 1): ", sum(results.built_PNA) , " / ", n_runs)


# Print a small table of the first 10 runs
show(first(results, min(10, n_runs)), allrows=true, allcols=true)


# ---------------------------------------
# Plots
# ---------------------------------------
# Plot 1: temperature vs % PN/A (scatter + loess smooth)
p1 = scatter(results.T, results.pct_PNA,
    xlabel = "Temperature (°C)", ylabel = "% flow routed to PN/A",
    title = "Temperature vs %PN/A (flow split chosen by MILP)",
    legend = false, markerstrokewidth=0.0, markersize=6)


# Add simple local regression / smoothing (moving average)
sort_idx = sortperm(results.T)
xs = results.T[sort_idx]
ys = results.pct_PNA[sort_idx]
# simple moving average smoothing
window = max(3, Int(round(n_runs/10)))
smooth_y = [mean(ys[max(1, j-window):min(length(ys), j+window)]) for j in 1:length(ys)]
plot!(p1, xs, smooth_y, lw=2, label="smoothed")


# Plot 2: temperature vs total cost
p2 = scatter(results.T, results.objective,
    xlabel = "Temperature (°C)", ylabel = "Objective cost (dollars/d approximate)",
    title = "Temperature vs Objective Cost",
    legend = false, markerstrokewidth=0.0, markersize=6)
plot!(p2, xs, [mean(results.objective[max(1, j-window):min(n_runs, j+window)]) for j in sort_idx], lw=2)


# Plot 3: temperature vs total GHG (kg CO2e / d)
p3 = scatter(results.T, results.total_GHG_CO2e_kg,
    xlabel = "Temperature (°C)", ylabel = "Total GHG (kg CO2e / d)",
    title = "Temperature vs Total GHG (kg CO2e / d)",
    legend = false, markerstrokewidth=0.0, markersize=6)
plot!(p3, xs, [mean(results.total_GHG_CO2e_kg[max(1, j-window):min(n_runs, j+window)]) for j in sort_idx], lw=2)


# Save plots
#=
savefig(p1, "temp_vs_pctPNA.png")
savefig(p2, "temp_vs_cost.png")
savefig(p3, "temp_vs_GHG.png")


println("\nPlots saved: temp_vs_pctPNA.png, temp_vs_cost.png, temp_vs_GHG.png")
println("If running in a REPL or notebook, plots should display automatically.")
=#
# ---------------------------------------
# Additional quick analytics
# ---------------------------------------
# How often does PN/A take > 50% of flow?
prop_pna_gt50 = count(x -> x > 50.0, results.pct_PNA) / n_runs
println("Fraction of runs with >50% flow to PN/A: ", round(prop_pna_gt50, digits=3))


# End
println("\nDone.")



# daily cost -> can change to yearly to look at the cost fluctuations throughout seasons
# graphs are completely wrong

#graphs: efficiency and economic side


  Activating project at `~/lab1-ec8771`
